In [ ]:
from pathlib import Path
from fastai.basics import *

In [ ]:
path=Path('./danbooru2020')
meta=path/'meta'

In [ ]:
def read_json_filter(p,dtype=False):
    table=pd.read_json(p,orient='records',typ='frame',lines=True,dtype={'id':'UInt32','image_width':'UInt16','image_height':'UInt16'})
    table[table['rating']=='s']
    return table[['id','image_width', 'image_height','tags']]

### Aggregate tags 

In [ ]:
def tags_by_file(table,aggregate_tags={}):
    for tags in table['tags']: 
        for tag in tags:
            if tag['name'] in aggregate_tags:
                aggregate_tags[tag['name']]['count']+=1
            else:
                aggregate_tags[tag['name']]={'id':tag['id'],'category':tag['category']}
                aggregate_tags[tag['name']]['count']=1
    return aggregate_tags
def tags_by_name(paths):
    named_tags={}
    for i,p in enumerate(paths):
        print(i)
        table = read_json_filter(p)
        named_tags=tags_by_file(table,named_tags)
    return named_tags

In [ ]:
list(meta.ls())

In [ ]:
named_tags=tags_by_name(meta.ls())

In [ ]:
named_tags

In [ ]:
with open('named_tags' + '.pkl', 'wb') as f: pickle.dump(named_tags, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
def tags_by_id(tags_list):
    return {tag['id']:name for name,tag in tags_list}
id_tags=tags_by_id(named_tags.items())

In [ ]:
id_tags

In [ ]:
with open('id_tags' + '.pkl', 'wb') as f: pickle.dump(id_tags, f, pickle.HIGHEST_PROTOCOL)

### Analyze Tag counts

In [ ]:
tag_counts=sorted(map(lambda t:t['count'],named_tags.values()))

In [ ]:
len(tag_counts)-len(list(filter(lambda t:t!=1,tag_counts)))

In [ ]:
tag_count=array(tag_counts)

In [ ]:
id_tags['470575'],len(id_tags)

In [ ]:
list(filter(lambda t:t['count']==3021593,named_tags.values()))

In [ ]:
named_tags

In [ ]:
list(filter(lambda i: i[1]['category'] == '4',named_tags.items()))

In [ ]:
array(tag_counts[134943:]),array(tag_counts[434500:])

In [ ]:
tag_counts[434000],tag_counts[-1]

Tags are definitely scewed to a smaller subset of more popular tags. 

In [ ]:
plt.yscale("log")
plt.plot(tag_counts[:])

### Fix datatypes

In [ ]:
named_tags,id_tags

In [ ]:
named_tags={k:{'id':int(v['id']),'category':int(v['category']),'count':int(v['count'])} for k,v in named_tags.items()}
with open('named_tags' + '.pkl', 'wb') as f: pickle.dump(named_tags, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
id_tags={int(k):v for k,v in id_tags.items()}
with open('id_tags' + '.pkl', 'wb') as f: pickle.dump(id_tags, f, pickle.HIGHEST_PROTOCOL)

### Concat Tables 

Restart notebook here if run out of memory

In [ ]:
with open('named_tags.pkl','rb') as f: named_tags=pickle.load(f)
with open('id_tags.pkl','rb') as f: id_tags=pickle.load(f)

In [ ]:
def numericalize_tags(tags_column):
    num_tags=[]
    for tags in tags_column:
        tgs=L(tags).map(lambda t:t['id']).map(int)
        tgs=pd.array(list(tgs),dtype='UInt32')
        num_tags+=[tgs]
    return num_tags
def concat_tables(paths):
    concat_table=None
    numeric_tags=[]
    for i,p in enumerate(paths):
        print(i,end=',')
        table = read_json_filter(p)
        if(concat_table is not None): pd.concat([concat_table,table.drop('tags', axis=1).copy()])
        else: concat_table=table.drop('tags', axis=1).copy()
        numeric_tags+=numericalize_tags(table['tags'])
    concat_table['tags']=pd.Series(numeric_tags)
    return concat_table

In [ ]:
list(meta.ls())

[Path('danbooru2020/2020000000000000'),
 Path('danbooru2020/2020000000000001'),
 Path('danbooru2020/2020000000000002'),
 Path('danbooru2020/2020000000000003'),
 Path('danbooru2020/2020000000000004'),
 Path('danbooru2020/2020000000000005'),
 Path('danbooru2020/2020000000000006'),
 Path('danbooru2020/2020000000000007'),
 Path('danbooru2020/2020000000000008'),
 Path('danbooru2020/2020000000000009'),
 Path('danbooru2020/2020000000000010'),
 Path('danbooru2020/2020000000000011'),
 Path('danbooru2020/2020000000000012'),
 Path('danbooru2020/2020000000000013'),
 Path('danbooru2020/2020000000000014'),
 Path('danbooru2020/2020000000000015')]

In [ ]:
full_table=concat_tables(meta.ls())

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,

In [ ]:
full_table.to_pickle('fulll_table.pkl')

In [ ]:
full_table['image_width'].dtype

dtype('uint16')

In [ ]:
full_table

,id,image_width,image_height,tags
0,263202,1042,1024,"[1821, 466061, 437490, 16751, 537684, 16867, 1976, 13804, 454933, 446622, 1707, 16613, 13197, 1287928, 566835, 8526, 464575, 15189, 4123, 89189]"
1,3014637,1191,1684,"[1409425, 470575, 475187, 376054, 6539, 87788, 3389, 8601, 466499, 687973, 10960, 658573, 1566, 4311, 469576, 1442807, 9294, 11906, 412368, 212816, 15675, 444]"
2,2872191,756,869,"[470575, 497007, 632214, 380350, 1231860, 1377427, 11813, 469576, 16750, 11429, 389378, 1242303, 1709, 574407, 11154]"
3,2873161,537,562,"[1821, 273, 449529, 579793, 419429, 194013, 452195, 380350, 1231860, 1377427, 469576, 566835, 584749, 383282, 403577, 1387590, 684639, 581144, 1310137, 1242303, 16509, 268819, 15080, 472154, 412368, 429, 12682, 1709]"
4,2872175,868,1228,"[470575, 9114, 1247173, 613194, 513837, 380350, 426491, 1566, 1231860, 1377427, 401601, 1681, 438623, 2355, 16509, 268819, 8672, 412368, 1882, 515193, 10644]"
...,...,...,...,...
265859,1957699,1500,2529,"[452549, 540830, 470575, 11716, 1245245, 448621, 1245189, 13200, 613194, 10959, 3389, 8601, 478849, 613200, 616524, 622137, 1566, 1345396, 1231860, 13197, 7820, 387214, 450702, 9294, 390035, 383282, 16509, 12590, 8830, 429, 465265, 467104, 7952, 4569, 1882, 270, 1323068, 626, 4152]"
265860,1661819,857,1167,"[540830, 399836, 1585391, 1245245, 1244517, 396859, 464546, 1747, 63, 1575, 6532, 513837, 2373, 8705, 1231860, 9474, 849557, 567948, 1681, 566835, 1245319, 448202, 2355, 8565, 125238, 12682, 1709, 438129, 1243225]"
265861,632933,782,1122,"[470575, 8601, 7525, 9672, 378317, 667868, 8889, 212816]"
265862,1419820,800,600,"[470575, 1376130, 2585, 478565, 13200, 580545, 3389, 8601, 4607, 464559, 511141, 8066, 380350, 2646, 379945, 1707, 995722, 13197, 469576, 667868, 2365, 490999, 464572, 8526, 484168, 412368, 1815, 212816, 462808, 547463, 1250039, 760228]"


We now have a way to access all relevant data in ram memory in a fairly data efficient way. Taking ~4 GB as opposed to 100+ GB. Converting the tags to unsigned ints as opposed to dictionaries of strings.